In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [2]:
### include useful folders
import sys

sys.path.append("../vendors/mtl_girnet/data_prep/")

import json
import h5py
import numpy as np
import glob
import random
import pandas as pd
import re
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt

# nltk
import nltk

# tokenizer
from twokenize import tokenizeRawTweetText as tokenize

# for a particular dataset
from xml.dom import minidom

In [3]:

### SemEval 2017 Task A

df = pd.read_csv("../data/datastories-semeval2017-task4/dataset/Subtask_A/4A-English/SemEval2017-task4-dev.subtask-A.english.INPUT.txt", sep="\t", header=None)

decode_map = {"negative": -1, "neutral": 0, "positive": 1}

df[1] = df[1].apply(lambda x: decode_map[x])
df[2] = df[2].apply(lambda x: tokenize(x))

data = map( lambda x :{'sentiment': x[1] , 'tokens': x[2] , 'text': ' '.join(x[2])} , df.to_numpy() )

en_semeval_17 = list(data)


### English-Spanish Code Mixed Data 

sents = {"N":-1 , "P" :1 , "NONE":0}

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_train.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_train = list(data)

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/cs-corpus-with-tweets_test.txt", encoding='utf-8').read().split("\n") 
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': sents[x[1]] , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )
en_es_wssa_data_test = list(data)

en_es_wssa_data = list(en_es_wssa_data_train) + list(en_es_wssa_data_test)

### Spanish Tweet Dataset

xmldoc = minidom.parse("../vendors/mtl_girnet/data_prep/data_cm_senti/general-tweets-train-tagged.xml")
tweets = xmldoc.getElementsByTagName('tweet')

sents = {"N":-1 , "P" :1 , "NEU":0 , 'NONE':0 , "P+" : 1 , "N+":-1 }


es_tass1_data = []

for i in range( len(tweets)-1) :
    if i == 6055:
        continue # bad jogar
    textt = tweets[i].getElementsByTagName('content')[0].childNodes[0].data
    words = tokenize( textt )
    sentiment = tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('value')[0].childNodes[0].data
    assert len(tweets[i].getElementsByTagName('polarity')[0].getElementsByTagName('entity'))==0
    es_tass1_data.append({'text':textt , 'tokens':words , 'sentiment': sents[sentiment] })

### Some english tweet data

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/twitter4242.txt", "r", encoding="utf-8",errors='ignore').read().split("\n")[1:-1]
data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

en_twitter_data = list(data)

### es2_twitter_data

data = open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_dev_complete.txt", encoding="utf-8").read().split("\n")[1:-1]
data += open("../vendors/mtl_girnet/data_prep/data_cm_senti/1600_tweets_test_average_complete.tsv", encoding="utf-8").read().split("\n")[1:-2]

data = map( lambda x : x.split("\t") , data )
data = map( lambda x :{'sentiment': int(np.sign(int(x[0])-int(x[1]))) , 'tokens': tokenize(x[2]) , 'text': x[2] } , data )

es2_twitter_data = list(data)

def get_y(data):
    from keras.utils import to_categorical
    y = []
    for row in data:
        y.append(int(row['sentiment']))
    y = to_categorical(y,num_classes=3)
    return y


print("Code-Mixed: en_es_wssa_data: %d" % len(en_es_wssa_data))
print("Spanish: es2_twitter_data: %d" % len(es2_twitter_data))
print("Spanish: es_tass1_data: %d" % len(es_tass1_data))
print("English: en_twitter_data: %d" % len(en_twitter_data))
# print("English: en_sentiment140: %d" %len(en_sentiment140))
en_es_y =  get_y(en_es_wssa_data)
en_es_y_train =  get_y(en_es_wssa_data_train)
en_es_y_test =  get_y(en_es_wssa_data_test)
es_twitter_y = get_y(es2_twitter_data)
es_tass_y = get_y(es_tass1_data)
en_twitter_y = get_y(en_twitter_data)
en_semeval_17_y = get_y(en_semeval_17)
# en_sentiment140_y = get_y(en_sentiment140)

Code-Mixed: en_es_wssa_data: 3062
Spanish: es2_twitter_data: 3202
Spanish: es_tass1_data: 7217
English: en_twitter_data: 4241


Using TensorFlow backend.


In [4]:
# len(en_semeval_17)

In [5]:
# sentiment_analysis = []

In [6]:
# data = en_es_wssa_data_train
# lang = "cm"
# for sent in data:
#     sentiment_analysis.append("\t".join([sent['text'], str(sent['sentiment']), lang, "\n"]))

In [7]:
# with open("train.txt", "w") as f:
#     f.writelines(sentiment_analysis)

In [8]:
from tensorflow.keras import backend as K


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
def get_class_weight(y):
    """
    Used from: https://stackoverflow.com/a/50695814
    TODO: check validity and 'balanced' option
    :param y: A list of one-hot-encoding labels [[0,0,1,0],[0,0,0,1],..]
    :return: class-weights to be used by keras model.fit(.. class_weight="") -> {0:0.52134, 1:1.adas..}
    """
    y_integers = np.argmax(y, axis=1)
    class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
    d_class_weights = dict(enumerate(class_weights))
    return d_class_weights

from keras import backend as K
from keras import losses

def loss_ordinal(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * losses.categorical_crossentropy(y_true, y_pred)

In [10]:
# ! pip install bpemb

In [11]:
from bpemb import BPEmb
multibpemb = BPEmb(lang="multi", vs=1000, dim=200)

Setting dim=300 for multilingual BPEmb
BPEmb fallback: multi from vocab size 1000 to 100000


paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [12]:
weight_matrix = multibpemb.vectors

In [13]:
multibpemb

BPEmb(lang=multi, vs=100000, dim=300)

In [13]:
max_len = 32
zero_vector = [0 for _ in range(300)]
def get_x(data_):
    x_  = []
    for sent in data_:
        pred = list(multibpemb.embed(sent['text']))
        if len(pred) >= 32:
            pred = pred[:32]
        else:
            counter = len(pred)
            while counter < max_len:
                pred.append(zero_vector)
                counter = counter + 1
        x_.append(pred)
    return np.array(x_)
en_es_x =  get_x(en_es_wssa_data)
es_twitter_x = get_x(es2_twitter_data)
es_tass_x = get_x(es_tass1_data)
en_twitter_x = get_x(en_twitter_data)
en_semeval_17_x = get_x(en_semeval_17)
en_es_x_train =  get_x(en_es_wssa_data_train)
en_es_x_test =  get_x(en_es_wssa_data_test)

In [14]:
# GET Y PADDED TOKENS NUMBER
max_len = 32
def get_x(data_):
    x_  = []
    for sent in data_:
        pred = list(multibpemb.encode_ids(sent['text']))
        if len(pred) >= 32:
            pred = pred[:32]
        else:
            counter = len(pred)
            while counter < max_len:
                pred.append(0)
                counter = counter + 1
        x_.append(pred)
    return np.array(x_)
en_es_x =  get_x(en_es_wssa_data)
es_twitter_x = get_x(es2_twitter_data)
es_tass_x = get_x(es_tass1_data)
en_twitter_x = get_x(en_twitter_data)
en_semeval_17_x = get_x(en_semeval_17)
en_es_x_train =  get_x(en_es_wssa_data_train)
en_es_x_test =  get_x(en_es_wssa_data_test)

In [9]:
import fasttext

In [10]:
embed = fasttext.load_model('../vendors/language-models/all_p_fasttext.bin')

In [11]:
max_len = 32
zero_vector = [0 for _ in range(100)]
def get_x(data_):
#     x_  = []
#     for sent in data_:
#         x_.append(embed.get_sentence_vector(sent['text'].replace("\n"," ")))
#     return np.array(x_)
    x_  = []
    for sent in data_:
        tokenised = fasttext.tokenize(sent['text'])
        sent_vector = []
        counter = 0
        for token in tokenised:
            if counter >= max_len:
                break
            else:
                sent_vector.append(embed[token])
                counter = counter + 1
        
        if counter < max_len:
            sent_vector.append(embed['</s>'])
            counter = counter + 1
                               
        while counter < max_len:
            sent_vector.append(zero_vector)
            counter = counter + 1
            
        x_.append(sent_vector)
        
    return np.array(x_)

en_es_x =  get_x(en_es_wssa_data)
es_twitter_x = get_x(es2_twitter_data)
es_tass_x = get_x(es_tass1_data)
en_twitter_x = get_x(en_twitter_data)
en_semeval_17_x = get_x(en_semeval_17)
en_es_x_train =  get_x(en_es_wssa_data_train)
en_es_x_test =  get_x(en_es_wssa_data_test)

In [12]:
from keras.layers import *
from keras.models import Sequential
from keras.preprocessing import sequence
from attention_lstm import AttentionWithContext
from keras.callbacks import *

In [ ]:
model = Sequential()
model.add(Dense(50, input_shape=(100,)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy', f1])

In [13]:
model = Sequential()
model.add(Bidirectional(LSTM(50, dropout=0.3, input_shape=(32, 100), recurrent_dropout=0.3, return_sequences=True)))
model.add(AttentionWithContext())
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# !pip install keras-self-attention
from keras_self_attention import SeqSelfAttention

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(50, input_shape=(None, 32, 100))))
# model.add(SeqSelfAttention(attention_activation='sigmoid'))
# model.add(Flatten())
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss=loss_ordinal,
              optimizer='adam',
              metrics=['accuracy', f1])

In [ ]:
# Convolution
kernel_size = 5
filters = 64
pool_size = 4

model = Sequential()
model.add(Conv1D(3,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1, input_shape=(32, 300)))
# model.add(MaxPooling1D(pool_size=pool_size))
# model.add(Conv1D(5,
#                  64,
#                  padding='valid',
#                  activation='relu',
#                  strides=1))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(50, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dropout(0.3))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', f1])

In [14]:
es = EarlyStopping(monitor='val_f1', mode='max', verbose=1, patience=5, restore_best_weights=True)

In [15]:
# history = model.fit(en_es_x_train, en_es_y_train, epochs=10, shuffle=True, validation_data=(en_es_x_test
#                                                                                             ,en_es_y_test))

In [16]:
# model.evaluate(X_test, y_test)

In [17]:
history = model.fit(en_semeval_17_x, en_semeval_17_y, epochs=20, initial_epoch=0, validation_split=0.2, shuffle=True, callbacks=[es])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 16505 samples, validate on 4127 samples
Epoch 1/20
16505/16505 [==============================] - 51s 3ms/step - loss: 0.8663 - acc: 0.5896 - f1: 0.5304 - val_loss: 0.8403 - val_acc: 0.5939 - val_f1: 0.5762
Epoch 2/20
16505/16505 [==============================] - 49s 3ms/step - loss: 0.7743 - acc: 0.6503 - f1: 0.6303 - val_loss: 0.7985 - val_acc: 0.6208 - val_f1: 0.6046
Epoch 3/20
16505/16505 [==============================] - 50s 3ms/step - loss: 0.7523 - acc: 0.6596 - f1: 0.6443 - val_loss: 0.7942 - val_acc: 0.6283 - val_f1: 0.6130
Epoch 4/20
16505/16505 [==============================] - 48s 3ms/step - loss: 0.7356 - acc: 0.6698 - f1: 0.6560 - val_loss: 0.7797 - val_acc: 0.6365 - val_f1: 0.6227
Epoch 5/20
16505/16505 [==============================] - 48s 3ms/step - loss: 0.7250 - acc: 0.6774 - f1: 0.6645 - val_l

In [18]:
model.evaluate(en_es_x_test, en_es_y_test)

613/613 [==============================] - 1s 1ms/step


[0.8596599811344707, 0.6084828711256117, 0.5769225707061917]

In [19]:
history = model.fit(es_tass_x, es_tass_y, epochs=35, initial_epoch=20, validation_split=0.2, shuffle=True, callbacks=[es])

Train on 5773 samples, validate on 1444 samples
Epoch 21/35
5773/5773 [==============================] - 17s 3ms/step - loss: 0.9434 - acc: 0.5486 - f1: 0.4771 - val_loss: 0.8299 - val_acc: 0.6191 - val_f1: 0.5794
Epoch 22/35
5773/5773 [==============================] - 17s 3ms/step - loss: 0.8495 - acc: 0.6099 - f1: 0.5604 - val_loss: 0.8071 - val_acc: 0.6343 - val_f1: 0.6043
Epoch 23/35
5773/5773 [==============================] - 17s 3ms/step - loss: 0.8191 - acc: 0.6300 - f1: 0.5851 - val_loss: 0.7876 - val_acc: 0.6447 - val_f1: 0.6063
Epoch 24/35
5773/5773 [==============================] - 17s 3ms/step - loss: 0.8024 - acc: 0.6392 - f1: 0.5900 - val_loss: 0.7725 - val_acc: 0.6503 - val_f1: 0.6140
Epoch 25/35
5773/5773 [==============================] - 17s 3ms/step - loss: 0.7906 - acc: 0.6451 - f1: 0.6024 - val_loss: 0.7719 - val_acc: 0.6503 - val_f1: 0.6174
Epoch 26/35
5773/5773 [==============================] - 17s 3ms/step - loss: 0.7839 - acc: 0.6461 - f1: 0.6083 - val_loss

In [20]:
model.evaluate(en_es_x_test, en_es_y_test)

613/613 [==============================] - 1s 1ms/step


[1.0563770489824926, 0.5138662318421032, 0.4927767060125827]

In [21]:
history = model.fit(es_twitter_x, es_twitter_y, epochs=40, initial_epoch=35, validation_split=0.2, shuffle=True,  callbacks=[es])

Train on 2561 samples, validate on 641 samples
Epoch 36/40
2561/2561 [==============================] - 8s 3ms/step - loss: 0.9673 - acc: 0.5517 - f1: 0.4960 - val_loss: 0.9054 - val_acc: 0.5788 - val_f1: 0.4871
Epoch 37/40
2561/2561 [==============================] - 8s 3ms/step - loss: 0.9028 - acc: 0.5814 - f1: 0.5019 - val_loss: 0.9065 - val_acc: 0.5913 - val_f1: 0.5097
Epoch 38/40
2561/2561 [==============================] - 8s 3ms/step - loss: 0.8752 - acc: 0.5955 - f1: 0.5485 - val_loss: 0.9091 - val_acc: 0.5835 - val_f1: 0.5320
Epoch 39/40
2561/2561 [==============================] - 8s 3ms/step - loss: 0.8612 - acc: 0.6076 - f1: 0.5536 - val_loss: 0.8794 - val_acc: 0.6147 - val_f1: 0.5320
Epoch 40/40
2561/2561 [==============================] - 7s 3ms/step - loss: 0.8454 - acc: 0.6107 - f1: 0.5671 - val_loss: 0.9165 - val_acc: 0.5803 - val_f1: 0.5443


In [22]:
model.evaluate(en_es_x_test, en_es_y_test)

613/613 [==============================] - 1s 998us/step


[0.8807512282547321, 0.579119086557267, 0.5189811075881099]

In [23]:
x = np.concatenate([en_semeval_17_x, en_twitter_x, es_tass_x, es_twitter_x])
y = np.concatenate([en_semeval_17_y, en_twitter_y, es_tass_y, es_twitter_y])

In [24]:
history = model.fit(x, y, epochs=60, initial_epoch=40,validation_data=(en_es_x_train,en_es_y_train), shuffle=True, callbacks=[es])

Train on 35292 samples, validate on 2449 samples
Epoch 41/60
35292/35292 [==============================] - 96s 3ms/step - loss: 0.7452 - acc: 0.6644 - f1: 0.6416 - val_loss: 0.8750 - val_acc: 0.5900 - val_f1: 0.5296
Epoch 42/60
35292/35292 [==============================] - 95s 3ms/step - loss: 0.7247 - acc: 0.6756 - f1: 0.6572 - val_loss: 0.8606 - val_acc: 0.5970 - val_f1: 0.5448
Epoch 43/60
35292/35292 [==============================] - 94s 3ms/step - loss: 0.7169 - acc: 0.6766 - f1: 0.6590 - val_loss: 0.8646 - val_acc: 0.5986 - val_f1: 0.5426
Epoch 44/60
35292/35292 [==============================] - 94s 3ms/step - loss: 0.7092 - acc: 0.6832 - f1: 0.6654 - val_loss: 0.8539 - val_acc: 0.6015 - val_f1: 0.5482
Epoch 45/60
35292/35292 [==============================] - 96s 3ms/step - loss: 0.7031 - acc: 0.6874 - f1: 0.6712 - val_loss: 0.8573 - val_acc: 0.6002 - val_f1: 0.5556
Epoch 46/60
35292/35292 [==============================] - 104s 3ms/step - loss: 0.7001 - acc: 0.6894 - f1: 0.6

In [25]:
model.evaluate(en_es_x_test, en_es_y_test)

613/613 [==============================] - 1s 1ms/step


[0.7836689443417124, 0.6574225123321447, 0.60803509546143]

In [26]:
history = model.fit(en_es_x_train, en_es_y_train, epochs=80, initial_epoch=60, validation_data=(en_es_x_test, en_es_y_test), shuffle=True,  callbacks=[es])

Train on 2449 samples, validate on 613 samples
Epoch 61/80
2449/2449 [==============================] - 8s 3ms/step - loss: 0.8355 - acc: 0.6174 - f1: 0.5727 - val_loss: 0.7602 - val_acc: 0.6639 - val_f1: 0.6314
Epoch 62/80
2449/2449 [==============================] - 8s 3ms/step - loss: 0.7964 - acc: 0.6497 - f1: 0.6239 - val_loss: 0.7387 - val_acc: 0.6591 - val_f1: 0.6467
Epoch 63/80
2449/2449 [==============================] - 8s 3ms/step - loss: 0.7695 - acc: 0.6697 - f1: 0.6378 - val_loss: 0.7397 - val_acc: 0.6639 - val_f1: 0.6494
Epoch 64/80
2449/2449 [==============================] - 8s 3ms/step - loss: 0.7449 - acc: 0.6823 - f1: 0.6531 - val_loss: 0.7454 - val_acc: 0.6672 - val_f1: 0.6457
Epoch 65/80
2449/2449 [==============================] - 8s 3ms/step - loss: 0.7258 - acc: 0.6750 - f1: 0.6584 - val_loss: 0.7283 - val_acc: 0.6786 - val_f1: 0.6673
Epoch 66/80
2449/2449 [==============================] - 8s 3ms/step - loss: 0.7268 - acc: 0.6880 - f1: 0.6710 - val_loss: 0.724

In [27]:
model.evaluate(en_es_x_test, en_es_y_test)

613/613 [==============================] - 1s 972us/step


[0.7347888143377631, 0.6737357259380098, 0.6703014879397817]

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

# MTL 

In [120]:
import tensorflow.keras as keras
from keras.layers import *
from keras.models import Model, Sequential
from attention_lstm import AttentionWithContext
from keras.callbacks import *
# from keras_self_attention import SeqSelfAttention

In [121]:
class GiretTwoCell(keras.layers.Layer):

    def __init__(self, cell_1 , cell_2 , nHidden , **kwargs):
        self.cell_1 = cell_1
        self.cell_2 = cell_2
        self.nHidden = nHidden
        self.state_size = [nHidden,nHidden]
        super(GiretTwoCell, self).__init__(**kwargs)

    def build(self, input_shape):
        
        nHidden = self.nHidden
        
        input_shape_n = ( input_shape[0] , input_shape[1]- 2 )
#         print "pp", input_shape_n
        
#         self.cell_1.build(input_shape_n)
#         self.cell_2.build(input_shape_n)
        
        self._trainable_weights += ( self.cell_1.trainable_weights )
        self._trainable_weights += ( self.cell_2.trainable_weights )
        
        self._non_trainable_weights += (  self.cell_1.non_trainable_weights )
        self._non_trainable_weights += (  self.cell_2.non_trainable_weights )
        
        self.built = True

    def call(self, inputs, states):
        
        nHidden = self.nHidden
        
        gate_val_1 = inputs[ : , 0:1]
        gate_val_2 = inputs[ : , 1:2]
        
        inputs  = inputs[ : , 2: ]
                
        gate_val_1 = K.repeat_elements(gate_val_1 , nHidden , -1 ) # shape # bs , hidden
        gate_val_2 = K.repeat_elements(gate_val_2 , nHidden , -1 ) # shape # bs , hidden
        
        _ , [h1 , c1 ]  = self.cell_1.call( inputs , states )
        _ , [h2 , c2 ]  = self.cell_2.call( inputs , states )
        
        h = gate_val_1*h1 + gate_val_2*h2  + (1 - gate_val_1 -  gate_val_2 )*states[0]
        c = gate_val_1*c1 + gate_val_2*c2  + (1 - gate_val_1 -  gate_val_2 )*states[1]
        
        return h, [h , c ]

In [170]:
hidden = 100
dims = 100

rnn_en = LSTM(hidden, name='en_lstm', recurrent_dropout=0.3, dropout=0.3)
rnn_hi = LSTM(hidden, name='es_lstm', recurrent_dropout=0.3, dropout=0.3)

       
# en
inp_en = Input(shape=(32, dims))
rnn_en_ = rnn_en(inp_en)
x = Dropout(0.3)(rnn_en_)
out_en = Dense(3, activation='softmax', name='en')(x)


# es
inp_hi = Input(shape=(32, dims))
rnn_hi_ = rnn_hi(inp_hi)
x = Dropout(0.3)(rnn_hi_)
out_hi = Dense(3, activation='softmax', name='es')(x)


cell_combined = GiretTwoCell(rnn_en.cell , rnn_hi.cell , hidden)

        
inp_enhi = Input(shape=(32, dims))
x = inp_enhi
x_att = x
x_att = Bidirectional(LSTM(32 , return_sequences=True, recurrent_dropout=0.3, dropout=0.3))( x )
bider_h = x_att 
x_att = Dropout(0.3)(x_att)
x_att = TimeDistributed(Dense(3, activation='softmax') )(x_att)
x_att = Lambda(lambda x : x[... , 1: ])(x_att)

x = Concatenate(-1)([x_att , x ])

x =  RNN(cell_combined, name='damn')(x)
# x = AttentionWithContext()(x)
out_enhi = Dense(3, activation='softmax', name='cm')(x)
        
model = Model( [inp_en , inp_hi , inp_enhi  ] , [ out_en , out_hi , out_enhi ] ) 

In [138]:
hidden = 64
numwords = weight_matrix.shape[0]
hidden_emd_dim = 300


embed = Embedding(numwords, hidden_emd_dim, weights=[weight_matrix])
conv1 = Conv1D(128, 3, activation='relu', padding='valid',strides=1)
pool1 = MaxPooling1D(2)
conv2 = Conv1D(128, 5, activation='relu', padding='valid', strides=1)
pool2 = MaxPooling1D(2)
# conv3 = Conv1D(128, 5, activation='relu', padding='valid', strides=1)
# pool3 = MaxPooling1D(2)
# conv3 = Conv1D(128, 5, activation='relu', padding='valid',strides=1)
# pool3 = MaxPooling1D(35)  # global max pooling


rnn_en = LSTM(hidden, name='en_lstm', recurrent_dropout=0.3, dropout=0.3)
rnn_hi = LSTM(hidden, name="hi_lstm", recurrent_dropout=0.3, dropout=0.3)
       
# en
inp_en = Input((None, ))
x = embed(inp_en)
x = conv1(x)
x = pool1(x)
x = conv2(x)
x = pool2(x)
# x = conv3(x)
# x = pool3(x)
x = Dropout(0.3)(x)
x = rnn_en(x)
out_en = Dense(3, activation='softmax')(x)


# es
inp_hi = Input((None, ))
x = embed(inp_hi)
x = conv1(x)
x = pool1(x)
x = conv2(x)
x = pool2(x)
# x = conv3(x)
# x = pool3(x)
x = Dropout(0.3)(x)
x = rnn_hi( x )
out_hi = Dense(3, activation='softmax')(x)


cell_combined = GiretTwoCell(rnn_hi.cell , rnn_en.cell , hidden)

inp_enhi = Input((None, ))
x = embed(inp_enhi)
x = conv1(x)
x = pool1(x)
x = conv2(x)
x = pool2(x)
# x = conv3(x)
# x = pool3(x)
x_att = x
x_att = Bidirectional(LSTM(32 , return_sequences=True, recurrent_dropout=0.3, dropout=0.3))( x )
bider_h = x_att 
x_att = Dropout(0.3)(x_att)
x_att = TimeDistributed(Dense(3, activation='softmax') )(x_att)
x_att = Lambda(lambda x : x[... , 1: ])(x_att)

x = Concatenate(-1)([x_att , x ])

x =  RNN(cell_combined, name='damn')(x)
# x = AttentionWithContext()(x)
out_enhi = Dense(3, activation='softmax', name='cm')(x)
        
model = Model( [inp_hi , inp_en , inp_enhi  ] , [ out_hi , out_en , out_enhi ] ) 

In [173]:
es = EarlyStopping(monitor='val_cm_f1', mode='max', verbose=1, patience=5, restore_best_weights=True)

In [174]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1])

In [175]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_48 (InputLayer)           (None, 32, 100)      0                                            
__________________________________________________________________________________________________
bidirectional_16 (Bidirectional (None, 32, 64)       34048       input_48[0][0]                   
__________________________________________________________________________________________________
dropout_44 (Dropout)            (None, 32, 64)       0           bidirectional_16[0][0]           
__________________________________________________________________________________________________
time_distributed_12 (TimeDistri (None, 32, 3)        195         dropout_44[0][0]                 
__________________________________________________________________________________________________
input_46 (

In [176]:
en_x = np.concatenate([en_twitter_x, en_semeval_17_x])
en_y = np.concatenate([en_twitter_y, en_semeval_17_y])
es_x = np.concatenate([es_twitter_x, es_tass_x])
es_y = np.concatenate([es_twitter_y, es_tass_y])

In [177]:
# all_x = np.concatenate([en_semeval_17_x, en_twitter_x, es_tass_x, es_twitter_x, en_es_x_train])
# all_y = np.concatenate([en_semeval_17_y, en_twitter_y, es_tass_y, es_twitter_y, en_es_y_train])
all_x = np.concatenate([en_semeval_17_x, es_tass_x, es_twitter_x])
all_y = np.concatenate([en_semeval_17_y, es_tass_y, es_twitter_y])

In [178]:
import random
def train_generator(batch_size=4, lang="cm"):
    b = batch_size
    
    while True:
        
#         n3 = np.random.randint(0, en_es_x_train.shape[0] - batch_size, batch_size)


        if lang is "cm": 
            n1 = np.random.randint(0, en_x.shape[0] - batch_size, batch_size)
            n2 = np.random.randint(0, es_x.shape[0] - batch_size, batch_size)
            p = random.random()
            if p < 0.3:
                p = random.random()
                if p < 0.5:
                    n3 = np.random.randint(0, en_x.shape[0] - batch_size, batch_size)
                    x = [en_x[n1,:], es_x[n2,:], en_x[n3,:]]
                    y = [en_y[n1,:], es_y[n2,:], en_y[n3,:]]
                else:
                    n3 = np.random.randint(0, es_x.shape[0] - batch_size, batch_size)
                    x = [en_x[n1,:], es_x[n2,:], es_x[n3,:]]
                    y = [en_y[n1,:], es_y[n2,:], es_y[n3,:]]
            else:
                n3 = np.random.randint(0, en_es_x_train.shape[0] - batch_size, batch_size)
                x = [en_x[n1,:], es_x[n2,:], en_es_x_train[n3,:]]
                y = [en_y[n1,:], es_y[n2,:], en_es_y_train[n3,:]]
        elif lang is "en":
            n3 = np.random.randint(0, en_x.shape[0] - batch_size, batch_size)
            x = [en_x[n3,:], en_x[n3,:], en_x[n3,:]]
            y = [en_y[n3,:], en_y[n3,:], en_y[n3,:]]
        elif lang is "es":
            n3 = np.random.randint(0, es_x.shape[0] - batch_size, batch_size)
            x = [es_x[n3,:], es_x[n3,:], es_x[n3,:]]
            y = [es_y[n3,:], es_y[n3,:], es_y[n3,:]]
        elif lang is "unsup":
            n1 = np.random.randint(0, en_x.shape[0] - batch_size, batch_size)
            n2 = np.random.randint(0, es_x.shape[0] - batch_size, batch_size)
            n3 = np.random.randint(0, all_x.shape[0] - batch_size, batch_size)
            x = [en_x[n1,:], es_x[n2,:], all_x[n3,:]]
            y = [en_y[n1,:], es_y[n2,:], all_y[n3,:]]
            
                 
        
#         x = [en_x[n1,:], es_x[n2,:], all_x[n3,:]]
#         y = [en_y[n1,:], es_y[n2,:], all_y[n3,:]]
#         x = [ en_es_x_train[n3,:],  en_es_x_train[n3,:], en_es_x_train[n3,:]]
#         y = [en_es_y_train[n3,:], en_es_y_train[n3,:], en_es_y_train[n3,:]]
        
        yield x, y

In [179]:
gen = train_generator(32, lang="en")
model.fit_generator(
    generator=gen,
    steps_per_epoch=200,
    epochs=20,
    initial_epoch=0,
    validation_data=([en_es_x_train,en_es_x_train,en_es_x_train],[en_es_y_train,en_es_y_train,en_es_y_train]),
    callbacks=[es]
)

Epoch 1/20
200/200 [==============================] - 92s 462ms/step - loss: 3.0285 - en_loss: 1.0161 - es_loss: 1.0112 - cm_loss: 1.0013 - en_f1: 0.2601 - es_f1: 0.2465 - cm_f1: 0.2419 - val_loss: 3.1544 - val_en_loss: 1.0589 - val_es_loss: 1.0706 - val_cm_loss: 1.0249 - val_en_f1: 0.2816 - val_es_f1: 0.3490 - val_cm_f1: 0.3096
Epoch 2/20
200/200 [==============================] - 66s 328ms/step - loss: 2.8154 - en_loss: 0.9592 - es_loss: 0.9626 - cm_loss: 0.8935 - en_f1: 0.4297 - es_f1: 0.4113 - cm_f1: 0.4940 - val_loss: 3.1506 - val_en_loss: 1.0754 - val_es_loss: 1.0658 - val_cm_loss: 1.0093 - val_en_f1: 0.3897 - val_es_f1: 0.3993 - val_cm_f1: 0.3921
Epoch 3/20
200/200 [==============================] - 67s 333ms/step - loss: 2.7342 - en_loss: 0.9403 - es_loss: 0.9365 - cm_loss: 0.8573 - en_f1: 0.4447 - es_f1: 0.4499 - cm_f1: 0.5355 - val_loss: 3.0093 - val_en_loss: 1.0412 - val_es_loss: 1.0173 - val_cm_loss: 0.9507 - val_en_f1: 0.3723 - val_es_f1: 0.4291 - val_cm_f1: 0.4606
Epoch 4

In [180]:
model.evaluate([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])

613/613 [==============================] - 2s 3ms/step


[2.6964681451900163,
 0.9170048700654682,
 0.9290861341537114,
 0.8503770978960283,
 0.4020865367053967,
 0.4387365517375131,
 0.5475156961607506]

In [181]:
gen = train_generator(32, lang="es")
model.fit_generator(
    generator=gen,
    steps_per_epoch=200,
    epochs=30,
    initial_epoch=20,
    validation_data=([en_es_x_train,en_es_x_train,en_es_x_train],[en_es_y_train,en_es_y_train,en_es_y_train]),
    callbacks=[es]
)

Epoch 21/30
200/200 [==============================] - 68s 338ms/step - loss: 2.9453 - en_loss: 1.0016 - es_loss: 1.0021 - cm_loss: 0.9417 - en_f1: 0.3142 - es_f1: 0.3501 - cm_f1: 0.4229 - val_loss: 2.8113 - val_en_loss: 0.9546 - val_es_loss: 0.9576 - val_cm_loss: 0.8991 - val_en_f1: 0.4130 - val_es_f1: 0.3947 - val_cm_f1: 0.4260
Epoch 22/30
200/200 [==============================] - 66s 329ms/step - loss: 2.7575 - en_loss: 0.9381 - es_loss: 0.9374 - cm_loss: 0.8820 - en_f1: 0.4716 - es_f1: 0.4631 - cm_f1: 0.4975 - val_loss: 2.8606 - val_en_loss: 1.0016 - val_es_loss: 0.9598 - val_cm_loss: 0.8992 - val_en_f1: 0.4059 - val_es_f1: 0.4529 - val_cm_f1: 0.4514
Epoch 23/30
200/200 [==============================] - 66s 332ms/step - loss: 2.7297 - en_loss: 0.9259 - es_loss: 0.9332 - cm_loss: 0.8706 - en_f1: 0.4866 - es_f1: 0.4762 - cm_f1: 0.5275 - val_loss: 2.7456 - val_en_loss: 0.9297 - val_es_loss: 0.9305 - val_cm_loss: 0.8853 - val_en_f1: 0.4051 - val_es_f1: 0.4212 - val_cm_f1: 0.4846
Epoc

In [182]:
model.evaluate([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])

613/613 [==============================] - 2s 3ms/step


[2.689742552709035,
 0.9178007408417458,
 0.9133689415202055,
 0.8585728392320978,
 0.40414127478980705,
 0.4419087814273492,
 0.49333386183951844]

In [183]:
gen = train_generator(32, lang="unsup")
model.fit_generator(
    generator=gen,
    steps_per_epoch=200,
    epochs=50,
    initial_epoch=30,
    validation_data=([en_es_x_train,en_es_x_train,en_es_x_train],[en_es_y_train,en_es_y_train,en_es_y_train]),
    callbacks=[es]
)

Epoch 31/50
200/200 [==============================] - 67s 334ms/step - loss: 2.4814 - en_loss: 0.8266 - es_loss: 0.8731 - cm_loss: 0.7817 - en_f1: 0.5843 - es_f1: 0.5364 - cm_f1: 0.6149 - val_loss: 2.7216 - val_en_loss: 0.9239 - val_es_loss: 0.9237 - val_cm_loss: 0.8740 - val_en_f1: 0.4299 - val_es_f1: 0.4880 - val_cm_f1: 0.5205
Epoch 32/50
200/200 [==============================] - 65s 323ms/step - loss: 2.4406 - en_loss: 0.8194 - es_loss: 0.8597 - cm_loss: 0.7615 - en_f1: 0.5757 - es_f1: 0.5603 - cm_f1: 0.6347 - val_loss: 2.7314 - val_en_loss: 0.9118 - val_es_loss: 0.9405 - val_cm_loss: 0.8791 - val_en_f1: 0.4788 - val_es_f1: 0.4437 - val_cm_f1: 0.5111
Epoch 33/50
200/200 [==============================] - 66s 328ms/step - loss: 2.4382 - en_loss: 0.8002 - es_loss: 0.8738 - cm_loss: 0.7642 - en_f1: 0.5996 - es_f1: 0.5347 - cm_f1: 0.6255 - val_loss: 2.6985 - val_en_loss: 0.8929 - val_es_loss: 0.9321 - val_cm_loss: 0.8735 - val_en_f1: 0.5155 - val_es_f1: 0.4354 - val_cm_f1: 0.5300
Epoc

In [184]:
model.evaluate([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])

613/613 [==============================] - 2s 3ms/step


[2.5449744120702076,
 0.850822454661763,
 0.8713245127368323,
 0.8228274669382739,
 0.5723932583802676,
 0.5343010656985334,
 0.5705737851766739]

In [185]:
gen = train_generator(32)
model.fit_generator(
    generator=gen,
    steps_per_epoch=200,
    epochs=75,
    initial_epoch=50,
    validation_data=([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])
)

Epoch 51/75
200/200 [==============================] - 62s 308ms/step - loss: 2.4788 - en_loss: 0.7693 - es_loss: 0.8659 - cm_loss: 0.8437 - en_f1: 0.6241 - es_f1: 0.5534 - cm_f1: 0.5659 - val_loss: 2.5210 - val_en_loss: 0.8522 - val_es_loss: 0.8809 - val_cm_loss: 0.7879 - val_en_f1: 0.5325 - val_es_f1: 0.5097 - val_cm_f1: 0.5923
Epoch 52/75
200/200 [==============================] - 62s 312ms/step - loss: 2.4723 - en_loss: 0.7903 - es_loss: 0.8596 - cm_loss: 0.8224 - en_f1: 0.6079 - es_f1: 0.5536 - cm_f1: 0.5903 - val_loss: 2.5234 - val_en_loss: 0.8709 - val_es_loss: 0.8868 - val_cm_loss: 0.7658 - val_en_f1: 0.5807 - val_es_f1: 0.4860 - val_cm_f1: 0.6427
Epoch 53/75
200/200 [==============================] - 63s 313ms/step - loss: 2.4573 - en_loss: 0.7997 - es_loss: 0.8523 - cm_loss: 0.8054 - en_f1: 0.6049 - es_f1: 0.5596 - cm_f1: 0.5991 - val_loss: 2.5012 - val_en_loss: 0.8501 - val_es_loss: 0.8950 - val_cm_loss: 0.7562 - val_en_f1: 0.5459 - val_es_f1: 0.4950 - val_cm_f1: 0.6439
Epoc

In [186]:
model.evaluate([en_es_x_test,en_es_x_test,en_es_x_test],[en_es_y_test,en_es_y_test,en_es_y_test])

613/613 [==============================] - 2s 3ms/step


[2.5119908143801166,
 0.8257767659221543,
 0.8894732844382282,
 0.7967407826887258,
 0.5720008315114368,
 0.5457861570316271,
 0.6657662629886708]

In [ ]:
#### ! pip3 install keras-self-attention

## Hindi English Dataset

In [ ]:
with open("../data/IIITH_Codemixed.txt") as f:
    lines = f.readlines()[:-1]
print(len(lines))

In [ ]:
sents = {"0":-1 , "1" :1 , "2":0}
data = map( lambda x : x.strip().split("\t") , lines )
data = map( lambda x :{'sentiment': sents[x[3]] , 'text': x[1] } , data )
hi_en_data = list(data)

In [ ]:
X_train = [x['text'] for x in hi_en_data]
y_train = [x['sentiment'] for x in hi_en_data]

In [ ]:
from keras.utils import to_categorical

In [ ]:
y_train = to_categorical(y_train,num_classes=3)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
max_len = 32
zero_vector = [0 for _ in range(300)]
def get_x(data_):
    x_  = []
    for sent in data_:
        pred = list(multibpemb.embed(sent))
        if len(pred) >= 32:
            pred = pred[:32]
        else:
            counter = len(pred)
            while counter < max_len:
                pred.append(zero_vector)
                counter = counter + 1
        x_.append(pred)
    return np.array(x_)
X_train =  get_x(X_train)
X_test = get_x(X_test)